In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import pickle
import clean_data
import random
import train
import json
from tqdm.notebook import tqdm

init_cols = ['BSTAR', 'INCLINATION', 'RA_OF_ASC_NODE', 'ECCENTRICITY', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY',
             'MEAN_MOTION', 'NORAD_CAT_ID', 'EPOCH', 'SUNSPOTS_1D', 'SUNSPOTS_3D', 'SUNSPOTS_7D',
             'AIR_MONTH_AVG_TEMP', 'WATER_MONTH_AVG_TEMP',
            ]


def load_raw(name=None):
    if not name:
        train_df = pd.read_pickle(os.environ['GP_HIST_PATH'] + '/../3_min/train.pkl' ) # Time: 25.7s
        test_df = pd.read_pickle(os.environ['GP_HIST_PATH'] + '/../3_min/test.pkl' ) # Time: 5
        return {'train': train_df, 'test': test_df}
    elif name == 'train':
        return {name: pd.read_pickle(os.environ['GP_HIST_PATH'] + '/../3_min/train.pkl' )}
    elif name == 'test':
        return {name: pd.read_pickle(os.environ['GP_HIST_PATH'] + '/../3_min/test.pkl' )}

def create_save(perc, name=None):
    #df_in = load_raw(name)
    
    if not name:
        names = ['train','test']
    else:
        names = [name]

    df_out = []
    #for name, df in df_in.items():
    for name in names:
        print(f'>>> Loading {name} raw data')
        df = pd.read_pickle(os.environ['GP_HIST_PATH'] + f'/../3_min/{name}.pkl' )[init_cols].reset_index(drop=True)
        
        print(f'>>> Normalizing {name} data...')
        df = clean_data.normalize_all_columns(df) # 53.4s

        print(f'>>> Building {name} index map...')
        try:
            idx_map = clean_data.load_index_map(name=name, path='data')
        except:
            idx_map = clean_data.create_index_map(df, write=True, name=name, path='data') # 3min 29s

        print(f'>>> Building {name} inputs and labels')
        X,y = clean_data.build_xy(df, idx_map) # 59min 41s
        X = clean_data.normalize_epoch_diff(X, drop_epoch=False) # 19s
        #all_X = []
        #all_y = []
        #batch_size = 1000000
        #batches = [idx_map[i:i+batch_size] for i in range(0, len(idx_map), batch_size)]
        #for batch in tqdm(batches):
        #    X,y = clean_data.build_xy(df, batch) # 59min 41s
        #    X = clean_data.normalize_epoch_diff(X, drop_epoch=False) # 19s
        #    all_X.append(X)
        #    all_y.append(y)
            
        #print(f'>>> Concate {name} all')
        #X = pd.concate(all_X)
        #y = pd.concate(all_y)

        print(f'>>> Saving {name} data')
        X.to_pickle(os.environ['GP_HIST_PATH'] + f'/cleaned/x_{name}1.pkl')
        y.to_pickle(os.environ['GP_HIST_PATH'] + f'/cleaned/y_{name}1.pkl')
        df_out.extend([X,y])
    
    return df_out

def load(perc, force_update=False):
    if not force_update:
        print('>>> Loading data')
        
        X_train = pd.read_pickle(os.environ['GP_HIST_PATH'] + '/cleaned/x_train1.pkl')
        y_train = pd.read_pickle(os.environ['GP_HIST_PATH'] + '/cleaned/y_train1.pkl')
        X_test = pd.read_pickle(os.environ['GP_HIST_PATH'] + '/cleaned/x_test1.pkl')
        y_test = pd.read_pickle(os.environ['GP_HIST_PATH'] + '/cleaned/y_test1.pkl')
        return X_train, y_train, X_test, y_test
    else:
        return create_save(perc)
        

perc = 0.05
force_update = False

X_train, y_train, X_test, y_test = load(perc, force_update)
print(f'>>> Complete')

>>> Loading data
>>> Complete


In [3]:
# configurations = {
#     'model_identifier' : "full_local_1",
#     'model_path' : './data',
#     'device' : 'cuda',
#     'random_seed' : 0,
#     'lr' : 0.1,
#     'momentum' : 0.9,
#     'weight_decay' : 1e-6,
#     'max_epochs' : 100,
#     'do_validate' : True,
#     'model_width' : 16,
#     'train_params' : {
#         'batch_size': 4000,
#         'shuffle': True,
#         'num_workers': 5,
#         'pin_memory': True,
#     },
#     'test_params' : {
#         'batch_size': 20000,
#         'num_workers': 5,
#         'pin_memory': True,
#     },
# }
# model_id=configurations['model_identifier']
# with open(f'data/{model_id}.cfg', 'w') as f:
#     json.dump(configurations, f)

In [4]:
model_id='full_local_1'

with open(f'data/{model_id}.cfg') as f:
    configurations = json.load(f)

In [ ]:
cols = ['BSTAR', 'INCLINATION', 'RA_OF_ASC_NODE', 'ECCENTRICITY',
       'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'MEAN_MOTION', 'SUNSPOTS_1D',
       'SUNSPOTS_3D', 'SUNSPOTS_7D', 'AIR_MONTH_AVG_TEMP',
       'WATER_MONTH_AVG_TEMP', 'year', 'month_sin', 'month_cos', 'hour_sin',
       'hour_cos', 'minute_sin', 'minute_cos', 'second_sin', 'second_cos',
       'ms_sin', 'ms_cos', 'year_y', 'month_sin_y', 'month_cos_y',
       'hour_sin_y', 'hour_cos_y', 'minute_sin_y', 'minute_cos_y',
       'second_sin_y', 'second_cos_y', 'ms_sin_y', 'ms_cos_y',
       'epoch_day_diff', 'epoch_sec_diff', 'epoch_ms_diff']

model, mean_losses = train.train_model(X_train[cols], y_train, X_test[cols], y_test, configurations, force_train=False)

Loading existing model
ResNet28(
  (stack1): ResNetStack(
    (stack): Sequential(
      (dense_1): ResnetDenseBlock(
        (dense1): Linear(in_features=37, out_features=16, bias=True)
        (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (dense2): Linear(in_features=16, out_features=16, bias=True)
        (bn2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dense3): Linear(in_features=16, out_features=16, bias=True)
        (bn3): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (dense_sc): Linear(in_features=37, out_features=16, bias=True)
          (bn_sc): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (identity_1a): ResnetIdentityBlock(
        (dense1): Linear(in_features=16, out_features=16, bias=True)
        (bn1): BatchNo

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/12193 [00:00<?, ?it/s]

  0%|          | 0/482 [00:00<?, ?it/s]

  0%|          | 0/12193 [00:00<?, ?it/s]

...

  0%|          | 0/12193 [00:00<?, ?it/s]

  0%|          | 0/482 [00:00<?, ?it/s]

  0%|          | 0/12193 [00:00<?, ?it/s]

  0%|          | 0/482 [00:00<?, ?it/s]

  0%|          | 0/12193 [00:00<?, ?it/s]